<a href="https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/16_Module_3_Notebook_VI_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Machine Learning and Linear/Logistic Regression

Let's consider **supervised** machine learning, which learns a function from input features to output classes.

## Autograder setup

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
%set_env HW_ID=cis2450_25f_HW9

In [ ]:
!pip3 install penngrader-client

In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

## Simple Linear Regression

Let's start with a really simple problem, which is fitting a line (linear function) to a set of points.

Here are the points...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Let's plot a series of points
X = 2 * np.random.rand(200, 1)

# We are plotting y = 2x + 3 plus a random
# value
y = 2 * X + 3 + np.random.randn(200, 1)

# Let's plot it
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(X, y)


Here's a simple machine learning model -- Linear Regression -- that finds the slope + intercept (sometimes called the weight + bias) for the best-fit line.

In [ ]:
from sklearn.linear_model import LinearRegression

# Use regression to get slope + intercept for
# a line matching the points
rgr = LinearRegression()
rgr.fit(X, y)

# Let's plot it
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(X, y)

# The line will be from 0 -> 3 non-inclusive
X2 = range(0,3)
y2 = rgr.coef_*X2 + rgr.intercept_
ax.plot(X2, y2.T, color='red')
plt.show()

print ("Intercept:", rgr.intercept_)
print ("Coefficient:",rgr.coef_)

How can we actually do this ourselves?

We can use the "normal equation" $(X^T X)^{-1} X^T y$ to compute this using a closed form solution.  It's a bit expensive, and it does require that we add the intercept $x_0=1$ to the $X$ matrix...

In [ ]:
from numpy.linalg import inv
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Insert the x_0 column
X_with_bias = np.ones((X.shape[0],2))
X_with_bias[:,1] = X.T

b = inv(X_with_bias.T.dot(X_with_bias)).dot(X_with_bias.T).dot(y)
print('Intercept: ', b[0])
print('Coefficient: ', b[1])

# predict using coefficients
yhat = X_with_bias.dot(b)
# plot data and predictions
# Let's plot it
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(X, y)

plt.plot(X, yhat, color='red')
plt.show()


## Linear Regression for Real Data


In [ ]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

housing_df = pd.DataFrame(data, columns=feature_names)
housing_df['Price'] = target
housing_df

In [ ]:
housing_df.describe()

In [ ]:
housing_df.hist(bins=50, figsize=(20,15))

In [ ]:
housing_df.info()

In [ ]:
y = target
X = data

In [ ]:
# Is the data correlated?
corr_matrix = housing_df.corr()

import seaborn as sn
import matplotlib.pyplot as plt

plt.figure(figsize=(15,8))
sn.heatmap(corr_matrix, annot=True)
plt.show()

Let's build a model!

First, maybe we should scale, decompose, etc.

Given the above correlation matrix, note there are high correlations between some of the fields. Maybe it's a good idea to use PCA to create new components.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Create our training and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# Normally we should scale and run PCA *only on the training data*, pretending
# we can't see the test data
scaler = StandardScaler()
X2_train = scaler.fit_transform(X_train)

# But we obviously need to transform as well
X2_test = scaler.transform(X_test)

pca = PCA(n_components=13)
X3_train = pca.fit_transform(X2_train)
X3_test = pca.transform(X2_test)

How much do the components contribute?

In [ ]:
pca.explained_variance_ratio_

Note that the last 2-3 components aren't super useful. We could easily reduce dimensionality to, say, 11 dimensions. But for now we'll keep things lossless.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

clf = LinearRegression()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

print("Score:", clf.score(X_test, y_test))
plt.figure(dpi=100)
plt.scatter(y_test, predictions)

# Different error measures
print("MAE:", mean_absolute_error(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))


In [ ]:
print(clf.intercept_)
print(clf.coef_)
plt.figure(dpi=100)
#plt.bar(housing_data.feature_names,clf.coef_)
plt.bar(['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12'], clf.coef_)

## Regularization

As a first step, we can use *lasso* or L1 regularization, which produces *sparse weights*.

In [ ]:
from sklearn.linear_model import Lasso

clf = Lasso()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

print("Score:", clf.score(X_test, y_test))

# Different error measures
print("MAE:", mean_absolute_error(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))

print(clf.coef_)
plt.figure(dpi=100)
#plt.bar(feature_names,clf.coef_)
plt.bar(['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12'], clf.coef_)

What about L2 (ridge)?

In [ ]:
from sklearn.linear_model import Ridge

clf = Ridge()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

print("Score:", clf.score(X_test, y_test))

# Different error measures
print("MAE:", mean_absolute_error(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))

print(clf.coef_)
plt.figure(dpi=100)
# plt.bar(feature_names,clf.coef_)
plt.bar(['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12'], clf.coef_)

And finally, what if we put the two together, in the form of Elastic Net?

In [ ]:
from sklearn.linear_model import ElasticNet


clf = ElasticNet()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

print("Score:", clf.score(X_test, y_test))

# Different error measures
print("MAE:", mean_absolute_error(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))

print(clf.coef_)
plt.figure(dpi=100)
# plt.bar(feature_names,clf.coef_)
plt.bar(['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12'], clf.coef_)

## Logistic Regression

Logistic regression uses techniques from linear regression but tries to map to Boolean values.

### The Logistic Function

Let's look at the **logit** or **logistic** function, also called the **sigmoid** function and indicated by $\sigma$, which is "near linear" but flattens at 0 and 1.  Let's look at several variations, also vs a linear and step function...

In [ ]:
X = [x * 0.01 for x in range(-400,400)]

def logit_1(x):
  return 1 / (1 + np.exp(-2*x))

def logit_2(x):
  return 1 / (1 + np.exp(-x))

def logit_3(x):
  return 1 / (1 + np.exp(-(x * 0.5)))


# Let's plot it
fig, ax = plt.subplots(figsize=(8,8))
ax.plot(X, [1 if x >= 0 else 0 for x in X], color='gray', label='x>=0')
ax.plot(X, [x/8+0.5 for x in X], color='brown', label='x/8 + 1/2')
ax.plot(X, [logit_1(x) for x in X], color='red', label='1/(1+exp(-2x))')
ax.plot(X, [logit_2(x) for x in X], color='blue', label='1/(1+exp(-x))')
ax.plot(X, [logit_3(x) for x in X], color='green', label='1/(1+exp(-x/2))')
ax.set_title('Boolean step vs linear vs logistic functions',fontsize='xx-large')
ax.set_xlim(-4, 4)
ax.set_ylim(-0.1, 1.1)
ax.legend(fontsize='x-large')
plt.tight_layout()

plt.show()


### Using Logistic Regression in SciKit-Learn

We can easily use a logistic regression classifier from SciKit to train and make predictions.

For simplicity, we'll start without scaling the data.

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_wine

dataset = load_wine()
dataset.feature_names

from sklearn.model_selection import train_test_split
import sklearn.metrics

X_train, X_test, y_train, y_test = train_test_split(dataset.data, \
                                                    dataset.target, \
                                                    test_size=0.40)

clf = LogisticRegression(max_iter=10000)
clf.fit(X_train,y_train)
prediction = clf.predict(X_test)

accuracy = sklearn.metrics.accuracy_score(prediction,y_test)
print("Accuracy: %.1f%%"% (accuracy*100))

Of course, we should scale the data first. We can create multi-stage *pipelines*...

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scl = StandardScaler()

pca = PCA()

clf = LogisticRegression(max_iter=10000, penalty='l2')

pipe = Pipeline(steps=[('Scale',scl),('PCA',pca),('LogReg',clf)])

pipe.fit(X_train,y_train)

prediction = pipe.predict(X_test)

accuracy = sklearn.metrics.accuracy_score(prediction,y_test)
print("Accuracy: %.1f%%"% (accuracy*100))

## Gradient Descent

Gradient descent consists of taking *steps* towards our optimal values (for weights).

For linear regression without regularization, there is a direct closed-form solution. For linear regression with regularization, or for logistic regression, we need to use gradient descent.

### Gradient Descent with a Linear Function

To look at gradient descent, let's first consider a simple example based on our wine dataset, with a linear value for our prediction.  (We'll relax this in a moment to consider the logistic function.)

We can define the cost function to be Mean Squared Error as follows:

$MSE = {1 \over n}\Sigma^n_{i=1}(\hat{y}^{(i)} - y^{(i)})$

where

$\hat{y}^{(i)} = \sigma(w^T \cdot x^{(i)})$.

(Recall $\sigma$ is the sigmoid function.)

For linear regression let's set up a simple line-fit:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Let's plot a series of points
X = 2 * np.random.rand(200, 1)

# We are plotting y = 2x + 3 plus a random
# value
y = 2 * X + 3 + np.random.randn(200, 1)

# Let's plot it
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(X, y)


Test and training sets...

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.metrics

# Change X-vector to include an additional feature with value 1,
# for the bias/intercept term

X2 = np.ones((X.shape[0],2))
X2[:,1] = X.T

X_train, X_test, y_train, y_test = train_test_split(X2, \
                                                    y, \
                                                    test_size=0.30)


Let's drill down to one of the features (dimensions) and see how the cost (weight) relates to the (squared) error of the overall cost function.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


x_coord = range(-5,15)

plt.figure(figsize=(12, 4))

w = np.random.randn(2)

# Here's our cost function, as mean squared error
def cost_fn(X, y, p, fn):
  if len(X.shape) == 1:
    # Single row, return the squared error
    return np.square(y - fn(X[p]))
  else:
    # Matrix, return the mean of the squared errors
    return np.average([cost_fn(X[i], y[i], p, fn) for i in range (0, X.shape[0])])

# Now let's plot the error function for a *single instance*, for each feature,
# for a range of weights and the *linear* function x*w
for feature in range(0, 2):
  ax = plt.subplot(1, 2, feature+1)
  ax.plot(x_coord, [cost_fn(X_train, y_train, feature, lambda x: x*w) for w in x_coord])
  ax.set_ylabel('Error for Feature ' + str(feature))


Since we have squared error, this should of course be a parabola!

The bottom of the parabola is the optimal weight for this particular dimension, and is what we are searching for.

Now imagine we are doing this simultaneously in all dimensions.

### Gradient Descent Function

The function takes a *step* eta, the weights $w$, and $X$ and $y$. We look at the overall error and step along each dimension.

In [ ]:
def gradient_descent(epochs, eta, X, w, y):
  """
  The gradient descent iterates for *epochs* rounds, making a step
  of size eta.  It will be adjusting w, based on the prediction for each
  instance vs y and the overall error.
  """
  # We'll use this list to accumulate
  # the error
  overall_error = []
  # Iterate over each epoch
  for i in range(epochs):
    # This is y-hat, the predictions for each
    # class label before they are thresholded
    # based on comparison with 0.5
    predictions = X.dot(w)

    # The overall error, as a vector
    error = (predictions - y)

    # Record the MSE so we can plot it
    mean_sq_error = np.sum(error ** 2) / X.shape[0]
    overall_error.append(mean_sq_error)

    # Now we update the weights.
    # The gradient is based on the partial derivative
    # of the MSE with respect to w.
    gradient = 2 / X.shape[0] * X.T.dot(error)

    w = w - eta * gradient

  return w, overall_error

# Training rounds or epochs
epochs = 500
eta = 0.01

w = np.zeros((2,1))
w[:,0] = np.random.randn(2)

weights, evect = gradient_descent(epochs, eta, X_train, w, y_train)
print (weights)
print (evect)

# Plot the mean-squared error
plt.plot(range(0,epochs), evect)
plt.xlabel('Gradient Descent Epoch #')
plt.ylabel('Mean-squared error (loss)')

## Gradient Descent for Training Logistic Regression

Let's try this with logistic regression, using the sigmoid function...

In [ ]:
from sklearn.datasets import make_blobs

(X, y) = make_blobs(n_samples=1000, n_features=2, centers=2, \
                    cluster_std=1.10, random_state=42)

import matplotlib.pyplot as plt


plt.scatter(x=X[:, 0], y=X[:, 1], c=y)

X_train, X_test, y_train, y_test = train_test_split(X, \
                                                    y, \
                                                    test_size=0.30)


First' let's look at the shape of the cost functions for $y_i=1$, $y_i=0$...  Recall that here we use *log loss*:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import math

def prediction(x):
  return 1.0 / (1 + np.exp(-x))

x = [(i - 500)/125 for i in range(1000)]

# Recall the cost function
plt.plot(x,[-math.log(prediction(i)) for i in x])
plt.plot(x,[-math.log(1 - prediction(i)) for i in x])

The derivative of the log loss function, over the sigmoid applied to the dot product of weights and instance, is almost identical to the derivative of the SSE function with linear regression.

In [ ]:
import numpy as np
from sklearn.metrics import log_loss

# Here is our sigmoid function for making
# predictions with logistic regression
# or with perceptron-style neural nets
def prediction(x):
  return 1.0 / (1 + np.exp(-x))


def gradient_descent(epochs, eta, X, w, y):
  """
  The gradient descent iterates for *epochs* rounds, making a step
  of size eta.  It will be adjusting w, based on the prediction for each
  instance vs y and the overall error.
  """
  # We'll use this list to accumulate
  # the error
  overall_error = []
  # Iterate over each epoch
  for i in range(epochs):
    # This is y-hat, the predictions for each
    # class label before they are thresholded
    # based on comparison with 0.5
    predictions = prediction(X.dot(w))

    # The overall error, as a vector
    error = (predictions - y)

    # Record the log loss so we can plot it
    mean_sq_error = log_loss(y, predictions)
    overall_error.append(mean_sq_error)

    # Now we update the weights.
    # The gradient is based on the partial derivative
    # of the log loss with respect to w.
    gradient = 1 / X.shape[0] * X.T.dot(error)

    w = w - eta * gradient

  return w, overall_error

# Training rounds or epochs
epochs = 500
eta = 0.001

w = np.random.randn(2)

weights, evect = gradient_descent(epochs, eta, X, w, y)
print (weights)
print (evect)

# Plot the mean-squared error
plt.plot(range(0,epochs), evect)
plt.xlabel('Gradient Descent Epoch #')
plt.ylabel('Log loss')

How close do we get to optimal? Let's plot where we set the weight, versus the minimum. Given the size of eta we won't necessarily get to the exact minimum.

In [ ]:
x_coord = range(-10,10)

plt.figure(figsize=(12, 4))

# Now let's plot the error function for each feature,
# for a range of weights and the output of the sigmoid function over x*w
for feature in range(0, 2):
  ax = plt.subplot(1, 2, feature+1)
  ax.plot(x_coord, [cost_fn(X_train, y_train, feature, lambda x: prediction(x*w)) for w in x_coord],
          marker='x')

  ax.axvline(x=weights[feature], color='red')
  ax.set_ylabel('Cost vs weight on feature ' + str(feature))
  ax.set_xlabel('Weight w' + str(feature))

Is it good enough to make predictions?

In [ ]:
for item in range(len(X_test)):
  predicted_label = 0 if prediction(X_test[item].dot(weights)) < 0.5 else 1

  print('Prediction {} vs {}'.format(predicted_label, y_test[item]))

## Exercise

Suppose we are training a Logistic Regression classifier.  Someone tells you your classifier is *overfitting* to the data.  You would want to:

In [ ]:
# TODO: pick an answer:
options = ['Increase L1 regularization',
           'Reduce L1 regularization']


my_option = options[] # TODO

In [ ]:
grader.grade('regularization', my_option)